In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("Train_Data.csv")

In [3]:
df.head()

,pc,ld,m0,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,m11,m12,m13,m14,ma,pred
0,y,83.71,63.0,7.2,52.5,14.0232,130.8300,12.4280,188.8425,8.9520,201.1905,9.2896,141.9075,16.0968,150.3390,12.4880,173.1240,m78,0
1,y,108.94,31.5,12.8,84.0,13.2840,128.8350,13.5256,183.0990,8.8520,207.2385,8.4704,154.7805,13.3304,101.0205,12.5096,131.4075,m78,0
2,E,169.65,0.0,5.6,73.5,14.5472,128.9295,13.6424,174.4680,8.9800,190.3125,11.3056,156.7650,NaN,122.5350,11.7136,176.8200,m76,1
3,x,122.42,31.5,7.2,63.0,15.0152,119.8575,12.3344,186.8580,10.7208,193.8195,10.6096,175.7490,NaN,124.8030,13.8424,168.2625,m55,0
4,E,125.43,94.5,7.2,42.0,14.4176,135.4290,14.5824,187.8135,9.3088,203.1540,9.8280,172.7040,14.4720,120.2145,NaN,150.1185,m76,0


## Cleaning data training


In [4]:
df['ma'].unique()

array(['m78', 'm76', 'm55', 'm75'], dtype=object)

In [5]:
encoded_ma = pd.get_dummies(df['ma'])
encoded_df = pd.get_dummies(df['pc'], prefix ='pc')

In [6]:
df_encoded = pd.concat([df,encoded_ma, encoded_df], axis=1)

In [7]:
df_encoded = df_encoded.drop(['ma','pc'], axis=1)

In [8]:
df_encoded

,ld,m0,m1,m2,m3,m4,m5,m6,m7,m8,...,pred,m55,m75,m76,m78,pc_E,pc_w,pc_x,pc_y,pc_z
0,83.71,63.0,7.2,52.5,14.0232,130.8300,12.4280,188.8425,8.9520,201.1905,...,0,0,0,0,1,0,0,0,1,0
1,108.94,31.5,12.8,84.0,13.2840,128.8350,13.5256,183.0990,8.8520,207.2385,...,0,0,0,0,1,0,0,0,1,0
2,169.65,0.0,5.6,73.5,14.5472,128.9295,13.6424,174.4680,8.9800,190.3125,...,1,0,0,1,0,1,0,0,0,0
3,122.42,31.5,7.2,63.0,15.0152,119.8575,12.3344,186.8580,10.7208,193.8195,...,0,1,0,0,0,0,0,1,0,0
4,125.43,94.5,7.2,42.0,14.4176,135.4290,14.5824,187.8135,9.3088,203.1540,...,0,0,0,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22579,139.08,73.5,7.2,84.0,13.8016,115.9725,13.8856,189.0000,8.6536,196.7490,...,1,0,0,1,0,1,0,0,0,0
22580,156.74,63.0,5.6,105.0,13.6464,130.2000,12.4688,180.3690,10.6984,212.6355,...,1,1,0,0,0,0,0,1,0,0
22581,115.63,147.0,4.8,31.5,13.4344,115.5000,13.5752,190.8900,9.6144,219.6810,...,0,0,0,0,1,0,1,0,0,0
22582,216.26,105.0,4.8,31.5,13.8176,112.5285,13.2408,166.8555,9.0160,200.3610,...,0,0,0,0,1,0,0,0,1,0


In [9]:
print(list(df_encoded.isnull().sum()))

[222, 0, 0, 0, 314, 467, 576, 675, 788, 900, 1047, 1100, 1227, 1381, 1499, 1589, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [10]:
columns_to_fill = []

for col in df_encoded.columns:
  if df_encoded[col].isnull().sum() != 0:
    columns_to_fill.append(col)


fill_nan_with_class_mean = lambda col: col.fillna(df_encoded.groupby('pred')[col.name].transform('mean'))
df_encoded[columns_to_fill] = df_encoded[columns_to_fill].apply(fill_nan_with_class_mean)

In [11]:
print(list(df_encoded.isnull().sum()))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [12]:
df_train = df_encoded

In [13]:
df_train['pred'].value_counts()

0    17711
1     4873
Name: pred, dtype: int64

In [14]:
df_encoded = pd.concat([df,encoded_ma, encoded_df], axis=1)

## Cleaning data testing


In [15]:
df2 = pd.read_csv('Test_Data.csv')

In [16]:
df2['ma'].unique()

array(['m76', 'm78', 'm75', 'm55'], dtype=object)

In [17]:
encoded_ma2 = pd.get_dummies(df2['ma'])
encoded_df2 = pd.get_dummies(df2['pc'], prefix ='pc')

In [18]:
df2_encoded = pd.concat([df2,encoded_ma2, encoded_df2], axis=1)

In [19]:
df2_encoded = df2_encoded.drop(['ma','pc'], axis=1)

In [20]:
df2_encoded

,index,ld,m0,m1,m2,m3,m4,m5,m6,m7,...,m14,m55,m75,m76,m78,pc_E,pc_w,pc_x,pc_y,pc_z
0,0,156.46,73.5,8.8,105.0,13.2768,113.5365,13.9552,168.5460,8.5896,...,166.1100,0,0,1,0,1,0,0,0,0
1,1,58.87,84.0,12.8,42.0,14.4440,126.6930,13.6688,190.6275,10.1712,...,177.7230,0,0,0,1,0,0,0,1,0
2,2,139.68,126.0,3.2,73.5,14.2432,107.4885,13.8544,168.3255,10.2504,...,185.6085,0,0,0,1,0,1,0,0,0
3,3,194.06,52.5,6.4,105.0,14.5480,138.6105,13.5080,196.6545,10.6304,...,180.2745,0,1,0,0,0,0,0,0,1
4,4,173.14,42.0,8.0,136.5,14.9496,103.8765,13.8936,168.3570,9.6184,...,157.6470,1,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3981,3981,109.45,126.0,4.0,94.5,14.6168,131.4495,13.1032,189.0840,8.3384,...,180.0435,0,1,0,0,0,0,0,0,1
3982,3982,188.31,136.5,1.6,21.0,12.9032,115.1535,14.3616,189.4830,7.6680,...,204.5505,0,0,0,1,0,1,0,0,0
3983,3983,80.99,31.5,4.8,115.5,15.0696,114.6810,14.0200,186.0810,9.2080,...,NaN,0,0,1,0,1,0,0,0,0
3984,3984,120.64,42.0,0.8,126.0,13.8960,123.6585,13.5496,187.8975,9.7944,...,164.2410,0,0,0,1,0,1,0,0,0


In [21]:
print(list(df2_encoded.isnull().sum()))

[0, 28, 0, 0, 0, 67, 71, 100, 121, 149, 148, 180, 200, 241, 220, 275, 285, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [22]:
df2_encoded = df2_encoded.drop('index', axis=1)

In [23]:
df2_encoded

,ld,m0,m1,m2,m3,m4,m5,m6,m7,m8,...,m14,m55,m75,m76,m78,pc_E,pc_w,pc_x,pc_y,pc_z
0,156.46,73.5,8.8,105.0,13.2768,113.5365,13.9552,168.5460,8.5896,187.6560,...,166.1100,0,0,1,0,1,0,0,0,0
1,58.87,84.0,12.8,42.0,14.4440,126.6930,13.6688,190.6275,10.1712,202.5870,...,177.7230,0,0,0,1,0,0,0,1,0
2,139.68,126.0,3.2,73.5,14.2432,107.4885,13.8544,168.3255,10.2504,214.2735,...,185.6085,0,0,0,1,0,1,0,0,0
3,194.06,52.5,6.4,105.0,14.5480,138.6105,13.5080,196.6545,10.6304,213.4650,...,180.2745,0,1,0,0,0,0,0,0,1
4,173.14,42.0,8.0,136.5,14.9496,103.8765,13.8936,168.3570,9.6184,191.2050,...,157.6470,1,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3981,109.45,126.0,4.0,94.5,14.6168,131.4495,13.1032,189.0840,8.3384,203.2275,...,180.0435,0,1,0,0,0,0,0,0,1
3982,188.31,136.5,1.6,21.0,12.9032,115.1535,14.3616,189.4830,7.6680,183.8865,...,204.5505,0,0,0,1,0,1,0,0,0
3983,80.99,31.5,4.8,115.5,15.0696,114.6810,14.0200,186.0810,9.2080,193.6200,...,NaN,0,0,1,0,1,0,0,0,0
3984,120.64,42.0,0.8,126.0,13.8960,123.6585,13.5496,187.8975,9.7944,189.0525,...,164.2410,0,0,0,1,0,1,0,0,0


In [24]:
col_means = df2_encoded.mean()
df2_test = df2_encoded.fillna(col_means)

## Formatting data

In [25]:
from sklearn.utils import resample
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
scaler = MinMaxScaler()
from sklearn import svm
import xgboost as xgb
import lightgbm as lgb

In [26]:
def resample_data(df_train):
  df_train_minority = df_train[df_train['pred'] ==1]
  df_train_majority = df_train[df_train['pred'] ==0]
  df_train_minority = resample(df_train_minority, replace = True, n_samples = len(df_train_majority), random_state = 0)
  return pd.concat([df_train_minority, df_train_majority])

In [27]:
df_resample = resample_data(df_train)

In [28]:
y_data = df_train['pred']
x_data = df_train.drop('pred',axis=1)

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.2, random_state = 0)

x_train = pd.DataFrame(scaler.fit_transform(x_train))
x_test = pd.DataFrame(scaler.transform(x_test))

In [29]:
y_data_res = df_resample['pred']
x_data_res = df_resample.drop('pred',axis=1)

x_train_res, x_test_res, y_train_res, y_test_res = train_test_split(x_data_res, y_data_res, test_size = 0.2, random_state = 0)

x_train_res = pd.DataFrame(scaler.fit_transform(x_train_res))
x_test_res = pd.DataFrame(scaler.transform(x_test_res))

In [30]:
df2 = pd.DataFrame(scaler.fit_transform(df2_test)) #test data final

In [31]:
print(y_train.value_counts())
print(y_train_res.value_counts())

0    14189
1     3878
Name: pred, dtype: int64
0    14180
1    14157
Name: pred, dtype: int64


In [32]:
def print_stats(df_x,df_y,clf):
    pred = clf.predict(df_x)
    pred = pd.DataFrame({'pred':pred})
    print(f"F1 score: {f1_score(df_y, pred)}")
    print(f"Accuracy: {accuracy_score(pred, df_y)}")
    # print(classification_report(df_y, pred))
    print(pred.value_counts())

## Logistic Regression

In [33]:
class_weights = {0: 1, 1: 4.63}
clf = LogisticRegression(class_weight=class_weights)
clf.fit(x_train, y_train)

print_stats(x_train, y_train, clf)
print("\n------------------\n")
print_stats(x_test, y_test, clf)

F1 score: 0.371177462075608
Accuracy: 0.4218187856312614
pred
1       12734
0        5333
dtype: int64

------------------

F1 score: 0.37356321839080464
Accuracy: 0.4208545494797432
pred
1       3181
0       1336
dtype: int64


In [34]:
clf = LogisticRegression()
clf.fit(x_train_res, y_train_res)

print_stats(x_train_res, y_train_res, clf)
print("\n------------------\n")
print_stats(x_test_res, y_test_res, clf)

F1 score: 0.539546655748688
Accuracy: 0.5634329674983237
pred
0       15627
1       12710
dtype: int64

------------------

F1 score: 0.5362448009506834
Accuracy: 0.5593507410021171
pred
0       3907
1       3178
dtype: int64


## SVM

In [35]:
# --> On Training and Validation data without Resample
class_weights = {0: 1, 1: 4.63}

clf2 = svm.SVC(kernel = 'rbf', C=10, gamma = 'auto',class_weight =class_weights)
clf2.fit(x_train, y_train)

print_stats(x_train, y_train, clf2)
print("\n------------------\n")
print_stats(x_test, y_test, clf2)

F1 score: 0.36089264648053476
Accuracy: 0.28034538108152984
pred
1       16466
0        1601
dtype: int64

------------------

F1 score: 0.3643122676579926
Accuracy: 0.2807172902368829
pred
1       4116
0        401
dtype: int64


In [36]:
# --> On Training and Validation data with Resample
clf = svm.SVC(kernel = 'rbf', C=10, gamma = 'auto')
clf.fit(x_train_res, y_train_res)

print_stats(x_train_res, y_train_res, clf)
print("\n------------------\n")
print_stats(x_test_res, y_test_res, clf)

F1 score: 0.5405198484094477
Accuracy: 0.5764195221794827
pred
0       16371
1       11966
dtype: int64

------------------

F1 score: 0.527479275406816
Accuracy: 0.565561044460127
pred
0       4125
1       2960
dtype: int64


In [37]:
# --> On Testing data
pred = clf2.predict(df2)
pred = pd.DataFrame({'pred': pred})
pred.value_counts()

pred
1       3663
0        323
dtype: int64

## Random forest

In [38]:
# --> On Training and Validation data without Resample

class_weights = {0: 1, 1: 4.63}
clf = RandomForestClassifier(n_estimators = 100,class_weight=class_weights,random_state=2)
clf.fit(x_train, y_train)

print_stats(x_train, y_train, clf)
print("\n------------------\n")
print_stats(x_test, y_test, clf)

F1 score: 1.0
Accuracy: 1.0
pred
0       14189
1        3878
dtype: int64

------------------

F1 score: 0.1501390176088971
Accuracy: 0.7969891520920965
pred
0       4433
1         84
dtype: int64


In [39]:
# --> On Training and Validation data with Resample

clf = RandomForestClassifier(n_estimators = 100,random_state=2)
clf.fit(x_train_res, y_train_res)

print_stats(x_train_res, y_train_res, clf)
print("\n------------------\n")
print_stats(x_test_res, y_test_res, clf)

F1 score: 1.0
Accuracy: 1.0
pred
0       14180
1       14157
dtype: int64

------------------

F1 score: 0.9715348304963526
Accuracy: 0.9719124911785463
pred
0       3648
1       3437
dtype: int64


In [40]:
# --> On Testing data
pred = clf.predict(df2)
pred = pd.DataFrame({'pred': pred})
pred.value_counts()

pred
0       3910
1         76
dtype: int64

## Gradient Boosting

In [41]:
# --> On Training and Validation data without Resample
clf2 = GradientBoostingClassifier()
clf2.fit(x_train, y_train)

print_stats(x_train, y_train, clf)
print("\n------------------\n")
print_stats(x_test, y_test, clf)

F1 score: 0.8714408973252804
Accuracy: 0.9505175181269718
pred
0       14991
1        3076
dtype: int64

------------------

F1 score: 0.8584320360970106
Accuracy: 0.9444321452291344
pred
0       3739
1        778
dtype: int64


In [42]:
# --> On Training and Validation data with Resample

clf = GradientBoostingClassifier()
clf.fit(x_train_res, y_train_res)

print_stats(x_train_res, y_train_res, clf)
print("\n------------------\n")
print_stats(x_test_res, y_test_res, clf)

F1 score: 0.7571851904744904
Accuracy: 0.7599957652539083
pred
0       14485
1       13852
dtype: int64

------------------

F1 score: 0.7441134751773049
Accuracy: 0.7453775582215949
pred
0       3589
1       3496
dtype: int64


In [43]:
# --> On Testing data
pred_gb = clf.predict(df2)
pred_gb = pd.DataFrame({'pred': pred_gb})
pred_gb.value_counts()

# --> this code is used to download prediction file
# file_path = 'pred_gb.csv'
# pred_gb.to_csv(file_path,index = False)
# from IPython.display import FileLink
# FileLink(file_path)

/content/pred_gb.csv

In [44]:
pred_gb.value_counts()

pred
0       2013
1       1973
dtype: int64

##XGBoost


In [46]:
# --> On Training and Validation data without Resample
clf2 = xgb.XGBClassifier()
clf2.fit(x_train, y_train)

print_stats(x_train, y_train, clf)
print("\n------------------\n")
print_stats(x_test, y_test, clf)

F1 score: 0.4401705682708207
Accuracy: 0.6439364587369236
pred
0       10454
1        7613
dtype: int64

------------------

F1 score: 0.41198759903548055
Accuracy: 0.6220943103829976
pred
0       2609
1       1908
dtype: int64


In [47]:
# --> On Training and Validation data with Resample
clf = xgb.XGBClassifier()
clf.fit(x_train_res, y_train_res)

print_stats(x_train_res, y_train_res, clf)
print("\n------------------\n")
print_stats(x_test_res, y_test_res, clf)

F1 score: 0.9538299737072743
Accuracy: 0.9535236616437873
pred
1       14368
0       13969
dtype: int64

------------------

F1 score: 0.8824725498169987
Accuracy: 0.8776287932251235
pred
1       3823
0       3262
dtype: int64


In [48]:
# --> On Testing data
x_gb = clf2.predict(df2)
x_gb = pd.DataFrame({'pred': x_gb})
x_gb.value_counts()

# --> this code is used to download prediction file
# file_path = 'X_gb.csv'
# x_gb.to_csv(file_path,index = False)
# from IPython.display import FileLink
# FileLink(file_path)

/content/X_gb.csv

In [49]:
x_gb.value_counts()

pred
0       3837
1        149
dtype: int64

##LightGBM

In [50]:
# --> On Training and Validation data without Resample
clf2 = lgb.LGBMClassifier()
clf2.fit(x_train, y_train)

print_stats(x_train, y_train, clf)
print("\n------------------\n")
print_stats(x_test, y_test, clf)

F1 score: 0.5724137931034483
Accuracy: 0.7940997398571982
pred
0       13245
1        4822
dtype: int64

------------------

F1 score: 0.5617977528089888
Accuracy: 0.7841487713083906
pred
0       3287
1       1230
dtype: int64


In [51]:
# --> On Training and Validation data with Resample
clf = lgb.LGBMClassifier()
clf.fit(x_train_res, y_train_res)

print_stats(x_train_res, y_train_res, clf)
print("\n------------------\n")
print_stats(x_test_res, y_test_res, clf)

F1 score: 0.895541090883776
Accuracy: 0.8940960581571796
pred
1       14572
0       13765
dtype: int64

------------------

F1 score: 0.8351618211738892
Accuracy: 0.8303458009880028
pred
1       3738
0       3347
dtype: int64


In [52]:
# --> On Testing data
l_gb = clf.predict(df2)
l_gb = pd.DataFrame({'pred': l_gb})
l_gb.value_counts()


# --> this code is used to download prediction file
# file_path = 'l_gb.csv'
# l_gb.to_csv(file_path,index = False)
# from IPython.display import FileLink
# FileLink(file_path)

/content/l_gb.csv

In [53]:
l_gb.value_counts()

pred
0       2523
1       1463
dtype: int64